In [6]:
import time
from concurrent.futures import ProcessPoolExecutor

import numpy as np
from pokerkit import (Automation, Card, Deck, FixedLimitTexasHoldem,
                      StandardHighHand, calculate_hand_strength, parse_range)
from tqdm import tqdm

In [7]:
# Create a game template (factory) that we can reuse
game = FixedLimitTexasHoldem(
    automations=(
        Automation.ANTE_POSTING,
        Automation.BET_COLLECTION,
        Automation.BLIND_OR_STRADDLE_POSTING,
        Automation.CARD_BURNING,
        Automation.HOLE_DEALING,
        Automation.BOARD_DEALING,
        Automation.HOLE_CARDS_SHOWING_OR_MUCKING,
        Automation.HAND_KILLING,
        Automation.CHIPS_PUSHING,
        Automation.CHIPS_PULLING,
    ),
    ante_trimming_status=True,  # use blinds instead of antes
    raw_antes=0,
    raw_blinds_or_straddles=(2, 4),
    small_bet=4,
    big_bet=8,
    starting_board_count=1,
)

In [8]:
# Simulate 10 hands with random actions
num_hands = 10
starting_stacks = 1000
player_count = 5
current_stacks = [starting_stacks] * player_count  # Track stacks across hands

for hand_num in tqdm(range(num_hands), desc="Simulating hands"):
    print(f"\n{'='*60}")
    print(f"HAND {hand_num + 1} - Button at position {hand_num % player_count}")
    print(f"{'='*60}")

    # Create a fresh state using the game factory
    state = game(
        raw_starting_stacks=current_stacks,
        player_count=player_count,
    )

    print(f"Starting stacks: {state.stacks}")
    print(f"\n--- HOLE CARDS ---")
    for i, hole_cards in enumerate(state.hole_cards):
        print(f"Player {i}: {hole_cards}")

    print(f"\n--- GAME PLAY ---")
    action_count = 0
    last_board_len = 0  # Track board length to detect changes

    # Play the hand with random actions
    while state.status:
        # Track board state changes and street
        board_len = len(state.board_cards)
        
        # Detect when board changes and print street/cards
        if board_len > last_board_len:
            if board_len == 3:
                board_str = " ".join([f"{str(card[0].rank)}{str(card[0].suit)}" for card in state.board_cards])
                print(f"\n*** FLOP *** [{board_str}]")
            elif board_len == 4:
                turn_card = f"{str(state.board_cards[3][0].rank)}{str(state.board_cards[3][0].suit)}"
                board_str = " ".join([f"{str(card[0].rank)}{str(card[0].suit)}" for card in state.board_cards])
                print(f"\n*** TURN *** [{turn_card}] (Board: {board_str})")
            elif board_len == 5:
                river_card = f"{str(state.board_cards[4][0].rank)}{str(state.board_cards[4][0].suit)}"
                board_str = " ".join([f"{str(card[0].rank)}{str(card[0].suit)}" for card in state.board_cards])
                print(f"\n*** RIVER *** [{river_card}] (Board: {board_str})")
            last_board_len = board_len
        
        # Determine current street for action display
        street_name = "PRE-FLOP"
        if board_len == 3:
            street_name = "FLOP"
        elif board_len == 4:
            street_name = "TURN"
        elif board_len == 5:
            street_name = "RIVER"

        # Check what actions are available
        if state.actor_indices:  # If there's an active player who needs to act
            available_actions = []

            # Check if folding is possible
            if state.can_fold():
                available_actions.append("fold")

            # Check if checking/calling is possible
            if state.can_check_or_call():
                available_actions.append("check_or_call")

            # Check if betting/raising is possible
            if state.can_complete_bet_or_raise_to():
                available_actions.append("bet_or_raise")

            # Choose a random action
            if available_actions:
                action = np.random.choice(available_actions)
                actor = state.actor_indices[0]

                # Print the action
                action_str = ""
                if action == "fold":
                    state.fold()
                    action_str = "folds"
                elif action == "check_or_call":
                    call_amount = state.checking_or_calling_amount
                    state.check_or_call()
                    action_str = (
                        "checks" if call_amount == 0 else f"calls {call_amount}"
                    )
                elif action == "bet_or_raise":
                    min_amount = state.min_completion_betting_or_raising_to_amount
                    state.complete_bet_or_raise_to()
                    action_str = f"bets/raises to {min_amount}"

                print(
                    f"[{street_name}] Player {actor}: {action_str} [Pot: {state.total_pot_amount}]"
                )
                action_count += 1
        else:
            # No player actions needed, state will auto-advance
            break

    print(f"\n--- FINAL BOARD ---")
    if len(state.board_cards) > 0:
        final_board = " ".join(
            [f"{str(card[0].rank)}{str(card[0].suit)}" for card in state.board_cards]
        )
    else:
        final_board = "EMPTY"
    print(f"Board: {final_board}")

    print(f"\n--- RESULTS ---")
    print(f"Final stacks: {state.stacks}")
    print(f"Payoffs: {state.payoffs}")

    # Show winning hands
    print(f"\nWinner at showdown:")
    for i, is_active in enumerate(state.statuses):
        if is_active:
            print(f"  Player {i}: {state.hole_cards[i]}")
    
    # Rotate stacks for next hand (button moves clockwise)
    # Move position 0 to the end, everyone else shifts left
    current_stacks = list(state.stacks[1:]) + [state.stacks[0]]

Simulating hands: 100%|██████████| 10/10 [00:00<00:00, 593.96it/s]


HAND 1 - Button at position 0
Starting stacks: [998, 996, 1000, 1000, 1000]

--- HOLE CARDS ---
Player 0: [6c, Ad]
Player 1: [8c, 5s]
Player 2: [5d, 2h]
Player 3: [7d, 4h]
Player 4: [4c, 7h]

--- GAME PLAY ---
[PRE-FLOP] Player 2: calls 4 [Pot: 10]
[PRE-FLOP] Player 3: calls 4 [Pot: 14]
[PRE-FLOP] Player 4: folds [Pot: 14]
[PRE-FLOP] Player 0: folds [Pot: 14]
[PRE-FLOP] Player 1: bets/raises to 8 [Pot: 18]
[PRE-FLOP] Player 2: folds [Pot: 18]
[PRE-FLOP] Player 3: folds [Pot: 0]

--- FINAL BOARD ---
Board: EMPTY

--- RESULTS ---
Final stacks: [998, 1010, 996, 996, 1000]
Payoffs: [-2, 10, -4, -4, 0]

Winner at showdown:
  Player 1: [8c, 5s]

HAND 2 - Button at position 1
Starting stacks: [1008, 992, 996, 1000, 998]

--- HOLE CARDS ---
Player 0: [6d, Qd]
Player 1: [Ac, 4c]
Player 2: [8h, Jh]
Player 3: [4s, 2h]
Player 4: [Js, 9s]

--- GAME PLAY ---
[PRE-FLOP] Player 2: folds [Pot: 6]
[PRE-FLOP] Player 3: calls 4 [Pot: 10]
[PRE-FLOP] Player 4: calls 4 [Pot: 14]
[PRE-FLOP] Player 0: calls 2

In [9]:
# Reusing executor for multiple calculations - saves overhead!
# Create executor once and reuse it for many calculations
sample_count = 100


class PokerEquityCalculator:
    def __init__(self):
        self._executor = ProcessPoolExecutor()

    def calculate_strength(
        self, hand, board, players_left_in_hand=5, sample_count=1000
    ):
        return calculate_hand_strength(
            player_count=4,
            hole_range=parse_range(hand),
            board_cards=Card.parse(board),
            hole_dealing_count=2,
            board_dealing_count=5,
            deck=Deck.STANDARD,
            hand_types=(StandardHighHand,),
            sample_count=sample_count,
            executor=self._executor,
        )

    def shutdown(self):
        if self._executor:
            self._executor.shutdown(wait=True)
            self._executor = None

    def __del__(self):
        self.shutdown()


equity_calculator = PokerEquityCalculator()

# Now do multiple calculations with the same executor
scenarios = [
    ("AsKs", "QsJs4h", "flush draw + straight draw"),
    ("AcAd", "Kh9s2c", "overpair on dry board"),
    ("7h6h", "8s9sTc", "straight on wet board"),
    ("7s2c", "AhKh10h", "junk hand vs over board w/flush possible"),
    ("QhQd", "QcQsAh10h9h", "quad queens - nuts"),
]

print("Calculating hand strengths (reusing executor)...\n")

results = []
for hand, board, description in scenarios:
    start = time.time()
    strength = equity_calculator.calculate_strength(
        hand, board, players_left_in_hand=3, sample_count=sample_count
    )
    elapsed = time.time() - start

    results.append((hand, board, description, strength, elapsed))

for hand, board, description, strength, elapsed in results:
    print(f"{hand} on {board} ({description})")
    print(f"  Strength: {strength:.1%} | Time: {elapsed:.2f}s")
    print()

Calculating hand strengths (reusing executor)...



0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0

AsKs on QsJs4h (flush draw + straight draw)
  Strength: 59.0% | Time: 0.55s

AcAd on Kh9s2c (overpair on dry board)
  Strength: 66.0% | Time: 0.02s

7h6h on 8s9sTc (straight on wet board)
  Strength: 59.9% | Time: 0.02s

7s2c on AhKh10h (junk hand vs over board w/flush possible)
  Strength: 4.0% | Time: 0.02s

QhQd on QcQsAh10h9h (quad queens - nuts)
  Strength: 100.0% | Time: 0.02s

